In [20]:
include("postgresql.jl")
include("TunnelingModule.jl")

##get training data
#8 combinations of handedness and ball type
fast_ll_train = get_data(true, islhp=true, islhb=true)
fast_lr_train = get_data(true, islhp=true, islhb=false)
fast_rl_train = get_data(true, islhp=false, islhb=true)
fast_rr_train = get_data(true, islhp=false, islhb=false)
slow_ll_train = get_data(true, islhp=true, islhb=true, fast=false)
slow_lr_train = get_data(true, islhp=true, islhb=false, fast=false)
slow_rl_train = get_data(true, islhp=false, islhb=true, fast=false)
slow_rr_train = get_data(true, islhp=false, islhb=false, fast=false)

##get test data
fast_ll_test = get_data(false, islhp=true, islhb=true)
fast_lr_test = get_data(false, islhp=true, islhb=false)
fast_rl_test = get_data(false, islhp=false, islhb=true)
fast_rr_test = get_data(false, islhp=false, islhb=false)
slow_ll_test = get_data(false, islhp=true, islhb=true, fast=false)
slow_lr_test = get_data(false, islhp=true, islhb=false, fast=false)
slow_rl_test = get_data(false, islhp=false, islhb=true, fast=false)
slow_rr_test = get_data(false, islhp=false, islhb=false, fast=false)

##calculate distances and ratios
fast_ll_train = calc_ratios(fast_ll_train)
fast_lr_train = calc_ratios(fast_lr_train)
fast_rl_train = calc_ratios(fast_rl_train)
fast_rr_train = calc_ratios(fast_rr_train)
slow_ll_train = calc_ratios(slow_ll_train)
slow_lr_train = calc_ratios(slow_lr_train)
slow_rl_train = calc_ratios(slow_rl_train)
slow_rr_train = calc_ratios(slow_rr_train)

##prep test data
fast_ll_test = calc_ratios(fast_ll_test)
fast_lr_test = calc_ratios(fast_lr_test)
fast_rl_test = calc_ratios(fast_rl_test)
fast_rr_test = calc_ratios(fast_rr_test)
slow_ll_test = calc_ratios(slow_ll_test)
slow_lr_test = calc_ratios(slow_lr_test)
slow_rl_test = calc_ratios(slow_rl_test)
slow_rr_test = calc_ratios(slow_rr_test)

Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
percent_lost = 0.7597173144876308
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
percent_lost = 0.9607701564380307
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
percent_lost = 0.947361168526939
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
percent_lost = 0.9553033066540735
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
percent_lost = 0.6058044519582921
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.
percent_lost = 0.8046996516396092
Connecting to pitch-tunneling database.
Data are stored in 'pitchtunnelingtest' and 'pitchtunnelingtrain' tables.

([5.69383 3.0249; 1.71071 0.740211; … ; 1.16038 0.283108; 2.26256 0.776059], [0.0376189, -0.0512863, -0.183523, -0.0599418, -0.0512863, 0.511992, -0.0629139, 0.0, 0.0410767, -0.145117  …  -0.0655811, -0.223813, 0.474374, -0.0512863, 0.0470651, -0.223813, 0.067862, 0.0470651, 0.0471762, -0.0599418])

We preprocess the data to be a tuple (X,Y) with X a matrix and Y a vector of runValues.<br>
The matrix will have two colums, the first being the ratio of release distance to tunnel distance and the second distance at the base to tunnel distance. Distances are calculated with respect to the mean release/tunnel/break point (in the x-z plane).

The dataset for fast balls and sinkers with both right-handed pitchers and batters is the largest set, so we will use this for timing of the algorithms.

In [24]:
include("TunnelingModule.jl")
##linear regression
@time linReg_fast_ll = lin_reg_pred(fast_ll_train[1],fast_ll_train[2],fast_ll_test[1])
linReg_fast_lr = lin_reg_pred(fast_lr_train[1],fast_lr_train[2],fast_lr_test[1])
linReg_fast_rl = lin_reg_pred(fast_rl_train[1],fast_rl_train[2],fast_rl_test[1])
@time linReg_fast_rr = lin_reg_pred(fast_rr_train[1],fast_rr_train[2],fast_rr_test[1])
linReg_slow_ll = lin_reg_pred(slow_ll_train[1],slow_ll_train[2],slow_ll_test[1])
linReg_slow_lr = lin_reg_pred(slow_lr_train[1],slow_lr_train[2],slow_lr_test[1])
linReg_slow_rl = lin_reg_pred(slow_rl_train[1],slow_rl_train[2],slow_rl_test[1])
linReg_slow_rr = lin_reg_pred(slow_rr_train[1],slow_rr_train[2],slow_rr_test[1])

##neural net
#getting rid of @time because it leads to "loss is NaN" error
m_fast_ll = nn_full(fast_ll_train[1],fast_ll_train[2])
m_fast_lr = nn_full(fast_lr_train[1],fast_lr_train[2])
m_fast_rl = nn_full(fast_rl_train[1],fast_rl_train[2])
m_fast_rr = nn_full(fast_rr_train[1],fast_rr_train[2])
m_slow_ll = nn_full(slow_ll_train[1],slow_ll_train[2])
m_slow_lr = nn_full(slow_lr_train[1],slow_lr_train[2])
m_slow_rl = nn_full(slow_rl_train[1],slow_rl_train[2])
m_slow_rr = nn_full(slow_rr_train[1],slow_rr_train[2])

#save parameters of nets
#save("nn_params.jld", "m_fast_ll", params(m_fast_ll), "m_fast_lr", params(m_fast_lr), "m_fast_rl", params(m_fast_rl), "m_fast_rr", params(m_fast_rr), "m_slow_ll", params(m_slow_ll), "m_slow_lr", params(m_slow_lr), "m_slow_rl", params(m_slow_rl), "m_slow_rr", params(m_slow_rr))
 
##Nearest neighbor classification
#k=16 seemed to give minimum mse in our tests
@time knn_fast_ll = knn_pred(fast_ll_train[1],fast_ll_train[2],fast_ll_test[1],16)
knn_fast_lr = knn_pred(fast_lr_train[1],fast_lr_train[2],fast_lr_test[1],16)
knn_fast_rl = knn_pred(fast_rl_train[1],fast_rl_train[2],fast_rl_test[1],16)
@time knn_fast_rr = knn_pred(fast_rr_train[1],fast_rr_train[2],fast_rr_test[1],16)
knn_slow_ll = knn_pred(slow_ll_train[1],slow_ll_train[2],slow_ll_test[1],16)
knn_slow_lr = knn_pred(slow_lr_train[1],slow_lr_train[2],slow_lr_test[1],16)
knn_slow_rl = knn_pred(slow_rl_train[1],slow_rl_train[2],slow_rl_test[1],16)
knn_slow_rr = knn_pred(slow_rr_train[1],slow_rr_train[2],slow_rr_test[1],16)

  0.017887 seconds (12.87 k allocations: 5.948 MiB)
  0.020326 seconds (48 allocations: 31.118 MiB, 33.24% gc time)


ErrorException: Loss is NaN

In [13]:
##read in model parameters
mpar_fast_ll = load("nn_params.jld", "m_fast_ll")
mpar_fast_lr = load("nn_params.jld", "m_fast_lr")
mpar_fast_rl = load("nn_params.jld", "m_fast_rl")
mpar_fast_rr = load("nn_params.jld", "m_fast_rr")
mpar_slow_ll = load("nn_params.jld", "m_slow_ll")
mpar_slow_lr = load("nn_params.jld", "m_slow_lr")
mpar_slow_rl = load("nn_params.jld", "m_slow_rl")
mpar_slow_rr = load("nn_params.jld", "m_slow_rr")

##neural net predictions
#from model parameters
@time m_fast_ll = nn_pred_params(mpar_fast_ll, fast_ll_test[1])
m_fast_lr = nn_pred_params(mpar_fast_lr, fast_lr_test[1])
m_fast_rl = nn_pred_params(mpar_fast_rl, fast_rl_test[1])
@time m_fast_rr = nn_pred_params(mpar_fast_rr, fast_rr_test[1])
m_slow_ll = nn_pred_params(mpar_slow_ll, slow_ll_test[1])
m_slow_lr = nn_pred_params(mpar_slow_lr, slow_lr_test[1])
m_slow_rl = nn_pred_params(mpar_slow_rl, slow_rl_test[1])
m_slow_rr = nn_pred_params(mpar_slow_rr, slow_rr_test[1])

#or directly from model
#@time m_fast_ll = nn_pred_orig(m_fast_ll, fast_ll_test[1])
#m_fast_lr = nn_pred_orig(m_fast_lr, fast_lr_test[1])
#m_fast_rl = nn_pred_orig(m_fast_rl, fast_rl_test[1])
#@time m_fast_rr = nn_pred_orig(m_fast_rr, fast_rr_test[1])
#m_slow_ll = nn_pred_orig(m_slow_ll, slow_ll_test[1])
#m_slow_lr = nn_pred_orig(m_slow_lr, slow_lr_test[1])
#m_slow_rl = nn_pred_orig(m_slow_rl, slow_rl_test[1])
#m_slow_rr = nn_pred_orig(m_slow_rr, slow_rr_test[1])

  0.147279 seconds (40.10 k allocations: 5.539 MiB)
  0.023191 seconds (96 allocations: 19.949 MiB, 16.32% gc time)


Tracked 60366×1 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
 -0.025446076978352022 
  0.001306785840404294 
 -0.03476926004417614  
 -0.020020799411235045 
 -0.003941012945185075 
 -0.055255180316160946 
  0.008663917181409126 
 -0.004965750566208072 
 -0.0017648925667192025
  7.672139834259662e-5 
  0.005400118417272076 
 -0.010797141534575082 
 -0.008121043593788032 
  ⋮                    
 -0.03429593582074291  
 -0.021821244887739358 
 -0.025615050273577293 
 -0.043107231577904276 
 -0.010930554596554494 
 -0.04266419178737391  
  0.00145480996099569  
 -0.04863366466096568  
 -0.019278253468911103 
 -0.018453195263203933 
  0.006013559529259202 
  0.005444425746747579 

In [14]:
##calculate metrics
sets = ["Fastball/sinkers - LL", "Fastball/sinkers - LR", "Fastball/sinkers - RL", "Fastball/sinkers - RR", "Other pitches - LL", "Other pitches - LR", "Other pitches - RL", "Other pitches - RR"]
mses_lR = [
    mse(linReg_fast_ll, fast_ll_test[2]),
    mse(linReg_fast_lr, fast_lr_test[2]),
    mse(linReg_fast_rl, fast_rl_test[2]),
    mse(linReg_fast_rr, fast_rr_test[2]),
    mse(linReg_slow_ll, slow_ll_test[2]),
    mse(linReg_slow_lr, slow_lr_test[2]),
    mse(linReg_slow_rl, slow_rl_test[2]),
    mse(linReg_slow_rr, slow_rr_test[2])
]
acc_decs_lR = [
    calc_acc_dec(linReg_fast_ll, fast_ll_test[2]),
    calc_acc_dec(linReg_fast_lr, fast_lr_test[2]),
    calc_acc_dec(linReg_fast_rl, fast_rl_test[2]),
    calc_acc_dec(linReg_fast_rr, fast_rr_test[2]),
    calc_acc_dec(linReg_slow_ll, slow_ll_test[2]),
    calc_acc_dec(linReg_slow_lr, slow_lr_test[2]),
    calc_acc_dec(linReg_slow_rl, slow_rl_test[2]),
    calc_acc_dec(linReg_slow_rr, slow_rr_test[2])
]
acc_ints_lR = [
    calc_acc_int(linReg_fast_ll, fast_ll_test[2]),
    calc_acc_int(linReg_fast_lr, fast_lr_test[2]),
    calc_acc_int(linReg_fast_rl, fast_rl_test[2]),
    calc_acc_int(linReg_fast_rr, fast_rr_test[2]),
    calc_acc_int(linReg_slow_ll, slow_ll_test[2]),
    calc_acc_int(linReg_slow_lr, slow_lr_test[2]),
    calc_acc_int(linReg_slow_rl, slow_rl_test[2]),
    calc_acc_int(linReg_slow_rr, slow_rr_test[2])
]
mses_m = [
    mse(m_fast_ll, fast_ll_test[2]),
    mse(m_fast_lr, fast_lr_test[2]),
    mse(m_fast_rl, fast_rl_test[2]),
    mse(m_fast_rr, fast_rr_test[2]),
    mse(m_slow_ll, slow_ll_test[2]),
    mse(m_slow_lr, slow_lr_test[2]),
    mse(m_slow_rl, slow_rl_test[2]),
    mse(m_slow_rr, slow_rr_test[2])
]
acc_decs_m = [
    calc_acc_dec(m_fast_ll, fast_ll_test[2]),
    calc_acc_dec(m_fast_lr, fast_lr_test[2]),
    calc_acc_dec(m_fast_rl, fast_rl_test[2]),
    calc_acc_dec(m_fast_rr, fast_rr_test[2]),
    calc_acc_dec(m_slow_ll, slow_ll_test[2]),
    calc_acc_dec(m_slow_lr, slow_lr_test[2]),
    calc_acc_dec(m_slow_rl, slow_rl_test[2]),
    calc_acc_dec(m_slow_rr, slow_rr_test[2])
]
acc_ints_m = [
    calc_acc_int(m_fast_ll, fast_ll_test[2]),
    calc_acc_int(m_fast_lr, fast_lr_test[2]),
    calc_acc_int(m_fast_rl, fast_rl_test[2]),
    calc_acc_int(m_fast_rr, fast_rr_test[2]),
    calc_acc_int(m_slow_ll, slow_ll_test[2]),
    calc_acc_int(m_slow_lr, slow_lr_test[2]),
    calc_acc_int(m_slow_rl, slow_rl_test[2]),
    calc_acc_int(m_slow_rr, slow_rr_test[2])
]
mses_knn = [
    mse(knn_fast_ll, fast_ll_test[2]),
    mse(knn_fast_lr, fast_lr_test[2]),
    mse(knn_fast_rl, fast_rl_test[2]),
    mse(knn_fast_rr, fast_rr_test[2]),
    mse(knn_slow_ll, slow_ll_test[2]),
    mse(knn_slow_lr, slow_lr_test[2]),
    mse(knn_slow_rl, slow_rl_test[2]),
    mse(knn_slow_rr, slow_rr_test[2])
]
acc_decs_knn = [
    calc_acc_dec(knn_fast_ll, fast_ll_test[2]),
    calc_acc_dec(knn_fast_lr, fast_lr_test[2]),
    calc_acc_dec(knn_fast_rl, fast_rl_test[2]),
    calc_acc_dec(knn_fast_rr, fast_rr_test[2]),
    calc_acc_dec(knn_slow_ll, slow_ll_test[2]),
    calc_acc_dec(knn_slow_lr, slow_lr_test[2]),
    calc_acc_dec(knn_slow_rl, slow_rl_test[2]),
    calc_acc_dec(knn_slow_rr, slow_rr_test[2])
]
acc_ints_knn = [
    calc_acc_int(knn_fast_ll, fast_ll_test[2]),
    calc_acc_int(knn_fast_lr, fast_lr_test[2]),
    calc_acc_int(knn_fast_rl, fast_rl_test[2]),
    calc_acc_int(knn_fast_rr, fast_rr_test[2]),
    calc_acc_int(knn_slow_ll, slow_ll_test[2]),
    calc_acc_int(knn_slow_lr, slow_lr_test[2]),
    calc_acc_int(knn_slow_rl, slow_rl_test[2]),
    calc_acc_int(knn_slow_rr, slow_rr_test[2])
]
DataFrame(Dataset=sets, LinRegMSE=mses_lR, NeuralNetMSE=mses_m, knnMSE=mses_knn, LinRegAccuracyDecimal=acc_decs_lR, NeuralNetAccuracyDecimal=acc_decs_m, knnAccuracyDecimal=acc_decs_knn, LinRegAccuracyInteger=acc_ints_lR, NeuralNetAccuracyInteger=acc_ints_m, knnAccuracyInteger=acc_ints_knn)


,Dataset,LinRegMSE,NeuralNetMSE,knnMSE,LinRegAccuracyDecimal,NeuralNetAccuracyDecimal,knnAccuracyDecimal,LinRegAccuracyInteger,NeuralNetAccuracyInteger,knnAccuracyInteger
,String,Float64,Flux…,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Fastball/sinkers - LL,0.0288437,0.0289978 (tracked),0.040706,30.0894,32.7015,29.1802,98.1661,98.1661,97.7115
2,Fastball/sinkers - LR,0.029455,0.0297943 (tracked),0.0436979,31.7838,33.9126,28.7848,98.1124,98.1183,97.5037
3,Fastball/sinkers - RL,0.0290889,0.0300713 (tracked),0.0440581,32.4426,37.6829,28.7973,98.0731,98.0731,97.3799
4,Fastball/sinkers - RR,0.0293848,0.0303066 (tracked),0.0424051,31.2148,36.5703,29.4981,97.9907,97.9959,97.5485
5,Other pitches - LL,0.0263634,0.0263286 (tracked),0.03786,36.3962,36.2909,32.4176,97.9581,97.9581,97.3687
6,Other pitches - LR,0.0275959,0.02855 (tracked),0.0405372,35.503,36.4609,30.502,97.8265,97.8265,97.2456
7,Other pitches - RL,0.0272089,0.0274955 (tracked),0.0427152,34.9953,37.1273,29.7055,98.0029,98.0029,97.2511
8,Other pitches - RR,0.0268113,0.0266076 (tracked),0.0400162,35.0893,35.5531,29.9771,97.9558,97.9575,97.3975
